# 0.0. Imports

In [1]:
#Import Libraries
import pandas as pd
import numpy as np

import streamlit as st

import plotly.express as px

import inflection

import sweetviz as sv
from ydata_profiling import ProfileReport

2023-04-11 21:53:14.206 INFO    visions.backends: Pandas backend loaded 1.4.2
2023-04-11 21:53:14.242 INFO    visions.backends: Numpy backend loaded 1.21.5
2023-04-11 21:53:14.272 INFO    visions.backends: Pyspark backend loaded 3.3.0
2023-04-11 21:53:14.273 INFO    visions.backends: Python backend loaded


# 0.1. Variables Guide

In [2]:
#Adjusting variables
COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}


COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

# 0.2. Functions

In [3]:
#Functions
#Adjusting float fata format
pd.set_option('display.float_format', '{:.2f}'.format)
np.set_printoptions(suppress=True)

In [4]:
#Show dataframe dimensions
def df_dimensions(df):
    print('Number of Rows: {}'.format(df.shape[0]))
    print('Number of Columns: {}'.format(df.shape[1]))
    
    return None

In [5]:
#Get numerical attributes
def get_num(df):
    return df.select_dtypes(include=['int64', 'float64'])

In [6]:
#Show first order statistics
def get_stats(df):
    # Central Tendency Metrics
    mean = pd.DataFrame(df.apply(np.mean)).T
    median = pd.DataFrame(df.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(df.apply(min)).T
    max_ = pd.DataFrame(df.apply(max)).T
    range_ = pd.DataFrame(df.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(df.apply(np.std)).T
    skew = pd.DataFrame(df.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(df.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return m

In [7]:
#Rename Columns
def rename_cols(df):
    df1 = df.copy()
    
    title = lambda x: inflection.titleize(x)
    spaces = lambda x: x.replace(' ', '') 
    snakecase = lambda x: inflection.underscore(x)
    
    cols_old = list(df1.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    
    cols_new = list(map(snakecase, cols_old))
    
    df1.columns = cols_new
    
    return df1

In [8]:
#Adjust country names
def country_name(country_id):
    return COUNTRIES[country_id]

In [9]:
#Adjust color names
def color_name(color_code):
    return COLORS[color_code]

In [10]:
#Adjust price type
def price_type(price_range):
    if price_range == 1:
        return 'Cheap'
    elif price_range == 2:
        return 'Normal'
    elif price_range == 3:
        return 'Expensive'
    else:
        return 'Gourmet'

In [11]:
#Adjust columns order
def adjust_cols_order(df):
    df1 = df.copy()

    new_cols_order = [
        'restaurant_id',
        'restaurant_name',
        'country',
        'city',
        'address',
        'locality',
        'locality_verbose',
        'longitude',
        'latitude',
        'cuisines',
        'price_type',
        'average_cost_for_two',
        'currency',
        'has_table_booking',
        'has_online_delivery',
        'is_delivering_now',
        'aggregate_rating',
        'rating_color',
        'color_name',
        'rating_text',
        'votes',
    ]
    
    return df1.loc[:, new_cols_order]

In [12]:
#Data processing
def process_data(df):
    df = df.dropna()

    df = rename_cols(df)

    df['price_type'] = df.loc[:, 'price_range'].apply(lambda x: price_type(x))

    df['country'] = df.loc[:, 'country_code'].apply(lambda x: country_name(x))

    df['color_name'] = df.loc[:, 'rating_color'].apply(lambda x: color_name(x))

    df['cuisines'] = df.loc[:, 'cuisines'].apply(lambda x: x.split(',')[0])

    df = df.drop_duplicates()

    df = adjust_cols_order(df)

    df.to_csv("base_restaurantes_tratada.csv", index=False)

    return df

# 1.0. Data Description

## 1.1. Loading Data

In [13]:
#Import data
df = pd.read_csv('base_restaurantes.csv')
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Italian,...,Botswana Pula(P),1,0,0,0,3,4.60,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Italian,...,Botswana Pula(P),1,0,0,0,3,4.60,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.70,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.40,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.03,14.57,American,...,Botswana Pula(P),0,0,0,0,3,4.40,5BA829,Very Good,858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.02,41.07,"Italian, Pizza, Fresh Fish",...,Turkish Lira(TL),0,0,0,0,4,4.30,5BA829,Very Good,1367
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.05,41.05,"Fast Food, Izgara, Seafood, Tea, Coffee",...,Turkish Lira(TL),0,0,0,0,2,4.50,3F7E00,Excellent,1172
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.05,41.05,Restaurant Cafe,...,Turkish Lira(TL),0,0,0,0,4,4.20,5BA829,Very Good,1160
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.02,41.03,"Home-made, Izgara",...,Turkish Lira(TL),0,0,0,0,3,4.00,5BA829,Very Good,770


## 1.2. Columns Description

## 1.2.1. Rename Columns

In [14]:
#Test Function Rename columns
df1 = rename_cols(df)
df1.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Italian,...,Botswana Pula(P),1,0,0,0,3,4.60,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Italian,...,Botswana Pula(P),1,0,0,0,3,4.60,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.70,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.40,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.03,14.57,American,...,Botswana Pula(P),0,0,0,0,3,4.40,5BA829,Very Good,858


## 1.3. Data Dimensions

In [15]:
#Check number of rows and columns of the dataframe
df_dimensions(df1)

Number of Rows: 7527
Number of Columns: 21


## 1.4. Check NA

In [16]:
#Check of NA's values os de dataframe
df1.isna().sum()

restaurant_id            0
restaurant_name          0
country_code             0
city                     0
address                  0
locality                 0
locality_verbose         0
longitude                0
latitude                 0
cuisines                15
average_cost_for_two     0
currency                 0
has_table_booking        0
has_online_delivery      0
is_delivering_now        0
switch_to_order_menu     0
price_range              0
aggregate_rating         0
rating_color             0
rating_text              0
votes                    0
dtype: int64

### 1.4.1. Drop / Replace NA's

In [17]:
#Drop / Replace NA's from dataframe
df1 = df1.dropna()
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

## 1.5. Data Dtypes

In [18]:
#Checking data types
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
dtype: object

## 1.6. Data Balancing

In [19]:
#Checking test data balance
df1['country_code'].value_counts(normalize=True)

1     0.47
216   0.19
215   0.06
189   0.05
214   0.04
30    0.03
148   0.03
14    0.02
37    0.02
208   0.02
166   0.01
162   0.01
191   0.01
184   0.01
94    0.01
Name: country_code, dtype: float64

In [20]:
COUNTRIES

{1: 'India',
 14: 'Australia',
 30: 'Brazil',
 37: 'Canada',
 94: 'Indonesia',
 148: 'New Zeland',
 162: 'Philippines',
 166: 'Qatar',
 184: 'Singapure',
 189: 'South Africa',
 191: 'Sri Lanka',
 208: 'Turkey',
 214: 'United Arab Emirates',
 215: 'England',
 216: 'United States of America'}

## 1.7. Pandas Profiling

In [21]:
#Report for External Data Analysis
prof_report = ProfileReport(df1)
prof_report.to_file(output_file='Profile_Report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 1.8. Sweet Viz Report

In [22]:
#Report for External Data Analysis
viz = sv.analyze(df1)
viz.show_html(filepath='SweetViz_Report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report SweetViz_Report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


## 1.9 Statistics

In [23]:
#Analysis of data statistics
num_data = get_num(df1)
get_stats(num_data)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.00,19040277.00,19039728.00,10543268.35,7701329.00,7075016.83,-0.08,-1.73
1,country_code,1.00,216.00,215.00,93.13,30.00,99.02,0.26,-1.85
2,longitude,-122.70,175.31,298.01,33.76,73.78,77.37,-0.68,-0.60
3,latitude,-41.33,55.98,97.31,19.31,25.24,23.27,-1.08,0.61
4,average_cost_for_two,0.00,25000017.00,25000017.00,7165.67,290.00,290877.34,84.49,7257.44
5,has_table_booking,0.00,1.00,1.00,0.06,0.00,0.24,3.62,11.13
6,has_online_delivery,0.00,1.00,1.00,0.35,0.00,0.48,0.61,-1.62
7,is_delivering_now,0.00,1.00,1.00,0.17,0.00,0.38,1.71,0.94
8,switch_to_order_menu,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,price_range,1.00,4.00,3.00,2.63,3.00,0.96,-0.04,-0.98


# 2.0. Business Analysis

In [24]:
df = process_data(df)
df

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,aggregate_rating,rating_color,color_name,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.01,14.45,Italian,...,1100,Botswana Pula(P),1,0,0,4.60,3F7E00,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,European,...,3100,Botswana Pula(P),0,0,0,4.70,3F7E00,darkgreen,Excellent,469
3,6301293,Banapple,Philippines,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.02,14.56,Filipino,...,800,Botswana Pula(P),0,0,0,4.40,5BA829,green,Very Good,867
4,6315689,Bad Bird,Philippines,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.03,14.57,American,...,700,Botswana Pula(P),0,0,0,4.40,5BA829,green,Very Good,858
5,6304833,Manam,Philippines,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.02,14.55,Filipino,...,700,Botswana Pula(P),0,0,0,4.70,3F7E00,darkgreen,Excellent,930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,Turkey,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.02,41.07,Italian,...,300,Turkish Lira(TL),0,0,0,4.30,5BA829,green,Very Good,1367
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,Turkey,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.05,41.05,Fast Food,...,45,Turkish Lira(TL),0,0,0,4.50,3F7E00,darkgreen,Excellent,1172
7524,5923535,Boon Cafe & Restaurant,Turkey,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.05,41.05,Restaurant Cafe,...,140,Turkish Lira(TL),0,0,0,4.20,5BA829,green,Very Good,1160
7525,5914190,Kanaat Lokantası,Turkey,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.02,41.03,Home-made,...,95,Turkish Lira(TL),0,0,0,4.00,5BA829,green,Very Good,770


In [25]:
#Verifying duplicates
#df2.duplicated()
#df2.drop_duplicates()

In [26]:
df.dtypes

restaurant_id             int64
restaurant_name          object
country                  object
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
price_type               object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
aggregate_rating        float64
rating_color             object
color_name               object
rating_text              object
votes                     int64
dtype: object

## 2.1. General Insights

### 1. Quantos restaurantes únicos estão registrados?

In [27]:
df['restaurant_id'].nunique()

6929

### 2. Quantos paises únicos estão registrados?

In [28]:
df['country'].nunique()

15

### 3. Quantas cidades únicas estão registradas?

In [29]:
df['city'].nunique()

125

### 4. Qual o total de avaliações feitas?

In [30]:
df['votes'].shape[0]

6929

### 5. Qual o total de tipos de culinária registrados?

In [31]:
df['cuisines'].nunique()

165

## 2.2. Countries

### 1. Qual o nome do país que possui mais cidades registradas?

In [32]:
df.loc[:, ['country', 'city']].groupby('country').count().sort_values('city', ascending=False).reset_index().iloc[0,0]

'India'

### 2. Qual o nome do país que possui mais restaurantes registrados?

In [33]:
df.loc[:, ['restaurant_id','country']].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'India'

### 3. Qual o nome do país que possui mais restaurantes com os nível de preço igual a 4 registrados?

In [34]:
df.loc[df['aggregate_rating']>=4, ['restaurant_id', 'country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'India'

### 4. Qual o nome da país que possui a maior quantidade de tipos de culinária distintos?

In [35]:
df.loc[:, ['country', 'cuisines']].groupby('country').nunique().sort_values('cuisines', ascending=False).reset_index().iloc[0,0]

'India'

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [36]:
df.loc[:, ['country', 'votes']].groupby('country').count().sort_values('votes', ascending=False).reset_index().iloc[0,0]

'India'

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [37]:
(df.loc[df['is_delivering_now']==1, ['country', 'restaurant_id']]
.groupby('country')
.count()
.sort_values('restaurant_id', ascending=False)
.reset_index()
.iloc[0,0])

'India'

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [38]:
cond = df['has_table_booking']==1
df.loc[cond, ['country', 'restaurant_id']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'India'

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [39]:
df.loc[:, ['country', 'votes']].groupby('country').mean().sort_values('votes', ascending=False).reset_index().iloc[0,0]

'Indonesia'

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [40]:
df.loc[:, ['country', 'aggregate_rating']].groupby('country').mean().sort_values('aggregate_rating', ascending=False).reset_index().iloc[0,0]

'Indonesia'

### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [41]:
df.loc[:, ['country', 'aggregate_rating']].groupby('country').mean().sort_values('aggregate_rating', ascending=True).reset_index().iloc[0,0]

'Brazil'

### 11. Qual a média de preço de um prato para dois por país?

In [42]:
df.loc[:, ['average_cost_for_two', 'country']].groupby('country').mean().sort_values('average_cost_for_two', ascending=False).reset_index()

,country,average_cost_for_two
0,Indonesia,303000.00
1,Australia,138959.78
2,Sri Lanka,2579.38
3,Philippines,1227.83
4,India,704.40
5,South Africa,339.23
6,Qatar,174.00
7,United Arab Emirates,153.72
8,Singapure,141.44
9,Brazil,138.81


## 2.2. Cities

### 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [43]:
df.loc[:, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Abu Dhabi'

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [44]:
cond = df['aggregate_rating']>4
df.loc[cond, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Bangalore'

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [45]:
cond =  df['aggregate_rating']<2.5
df.loc[cond, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Gangtok'

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [46]:
#df.loc[:, ['city', 'average_cost_for_two']].sort_values('average_cost_for_two', ascending=False).iloc[0,0]
df.loc[:, ['city', 'average_cost_for_two']].groupby('city').mean().sort_values('average_cost_for_two', ascending=False).reset_index().iloc[0,0]

'Adelaide'

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [47]:
df.loc[:, ['city', 'cuisines']].groupby('city').nunique().sort_values('cuisines', ascending=False).reset_index().iloc[0,0]

'Birmingham'

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [48]:
df.loc[df['has_table_booking']==1, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Bangalore'

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [49]:
df.loc[df['is_delivering_now']==1, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Vadodara'

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [50]:
cond= df['has_online_delivery']==1
df.loc[cond, ['city', 'restaurant_id']].groupby('city').nunique().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'Bhopal'

## 2.2. Restaurants

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [51]:
df.loc[:, ['restaurant_name', 'restaurant_id', 'votes']].sort_values(['votes', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Bawarchi'

### 2. Qual o nome do restaurante com a maior nota média?

In [52]:
df.loc[:, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Indian Grill Room'

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [53]:
cols = ['restaurant_name',  'restaurant_id', 'average_cost_for_two']
df.loc[:, cols].sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

"d'Arry's Verandah Restaurant"

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [54]:
cond = df['cuisines']=='Brazilian'
cols = ['restaurant_name', 'restaurant_id', 'aggregate_rating']
df.loc[cond, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'Loca Como tu Madre'

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [55]:
cond = (df['cuisines']=='Brazilian') & (df['country']=='Brazil')
cols = ['restaurant_name', 'restaurant_id', 'aggregate_rating']
df.loc[cond, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Braseiro da Gávea'

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [56]:
df.loc[:, ['has_online_delivery', 'votes']].groupby(['has_online_delivery']).mean().sort_values('votes', ascending=False).reset_index()

,has_online_delivery,votes
0,1,838.82
1,0,479.42


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [57]:
df.loc[:, ['has_table_booking', 'average_cost_for_two']].groupby('has_table_booking').mean().sort_values('average_cost_for_two', ascending=False).reset_index()

,has_table_booking,average_cost_for_two
0,1,69998.42
1,0,3488.60


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [58]:
cond = ((df['cuisines']=='Japanese') | (df['cuisines']=='BBQ')) & (df['country']=='United States of America')
df.loc[cond, ['cuisines', 'average_cost_for_two']].groupby('cuisines').mean().sort_values('average_cost_for_two', ascending=False).reset_index()

,cuisines,average_cost_for_two
0,Japanese,56.41
1,BBQ,39.64


## 2.3. Cousine

###  1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [59]:
cond = df['cuisines']=='Italian'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Darshan'

### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [60]:
cond = df['cuisines']=='Italian'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'Avenida Paulista'

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [61]:
cond = df['cuisines']=='American'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Burger & Lobster'

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [62]:
cond = df['cuisines']=='American'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'Alston Bar & Beef'

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [63]:
cond = df['cuisines']=='Arabian'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Mandi@36'

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [64]:
cond = df['cuisines']=='Arabian'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'Raful'

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [65]:
cond = df['cuisines']=='Japanese'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Sushi Samba'

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [66]:
cond = df['cuisines']=='Japanese'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'Banzai Sushi'

### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [67]:
cond = df['cuisines']=='Home-made'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,0]

'Kanaat Lokantası'

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [68]:
cond = df['cuisines']=='Home-made'
df.loc[cond, ['restaurant_name', 'restaurant_id', 'aggregate_rating']].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,0]

'GurMekan Restaurant'

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [69]:
df.loc[:,['cuisines', 'average_cost_for_two']].groupby('cuisines').max().sort_values('average_cost_for_two', ascending=False).reset_index().iloc[0,0]

'Modern Australian'

### 12. Qual o tipo de culinária que possui a maior nota média?

In [70]:
df.loc[:, ['cuisines', 'aggregate_rating']].groupby('cuisines').max().sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,cuisines,aggregate_rating
0,Indian,4.90


### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [71]:
cond = (df['has_online_delivery']==1) & (df['is_delivering_now']==1)
df.loc[cond, ['cuisines', 'restaurant_id']].groupby('cuisines').count().sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'North Indian'